In [2]:
import pandas as pd
import re

from langdetect import detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException

In [3]:
dessi_train_labels = pd.read_csv("DeSSI_v2/train_labels.csv")
dessi_train = pd.read_csv("DeSSI_v2/train.csv")

dessi_dev_labels = pd.read_csv("DeSSI_v2/dev_labels.csv")
dessi_dev = pd.read_csv("DeSSI_v2/dev.csv")

dessi_test_labels = pd.read_csv("DeSSI_v2/test_labels.csv")
dessi_test = pd.read_csv("DeSSI_v2/test.csv")

dessi_all = pd.concat([dessi_train, dessi_dev, dessi_test], axis=1)
dataset_split = ["train"] * dessi_train.shape[1] + ["validation"] * dessi_dev.shape[1] + ["test"] * dessi_test.shape[1]
dataset_split_df = pd.DataFrame(dataset_split).T
dataset_split_df.columns = dessi_all.columns
dessi_all = pd.concat([dessi_all, dataset_split_df]).reset_index(drop=True)

dessi_all_labels = pd.concat([dessi_train_labels, dessi_dev_labels, dessi_test_labels], axis=0).reset_index(drop=True)
dataset_split_df = pd.DataFrame(dataset_split)
dataset_split_df.columns = ["split_type"]
dessi_all_labels = pd.concat([dessi_all_labels, dataset_split_df], axis=1).reset_index(drop=True)

C:\Users\Luca\AppData\Local\Temp\ipykernel_1456\3396044622.py:2: DtypeWarning: Columns (62,107,161,241,255,324,449,503,686,720,807,812,889,934,1073,1113,1133,1154,1158,1242,1317,1333,1394,1408,1470,1553,1577,1674,1729,1762,1872,1913,1940,2045,2131,2139,2209,2288,2404,2436,2601,2627,2663,2779,2841,2960,2994,3061,3063,3080,3129,3136,3250,3319,3439,3464,3499,3520,3596,3637,3649,3748,3778,3787,3808,3902,3907,3917,3951,4009,4034,4037,4069,4129,4210,4218,4252,4325,4359,4386,4479,4646,4649,4652,4710,4936,4939,5013,5065,5069,5348,5352,5356,5399,5476,5702,5736,5889,5962,6072,6094,6107,6140,6159,6337,6360,6383,6407,6415,6423,6507,6555,6567,6590,6593,6689,6809,6815,6920,6995,7042,7127,7163,7229,7295,7301,7319,7321,7354,7399,7403,7465,7474,7593,7594,7694,7719,7798,7872,7979,8179,8298,8308,8473,8542,8565,8568,8570,8579,8633,8662,8707,8835,8846,8859,8899,8933,9024,9084,9126,9170,9184,9198,9251,9329,9412,9505,9588,9592,9610,9652,9803,9818,9907,9908,9952,9975,10073,10099,10136,10161,10359,10369,10407,

# Why DeSSI is not suitable for binary classification of personal data

1. The number of false labeled columns can be higher than expected
2. Dessi contains multiple columns which are labeled as other data, but contain personal-related Information (password, married, salary ...). To use the data all columns have to be manually labeled as personal or non-personal which will take up to much time for this project. It can not be clearly differed into personal data and non-personal data by the labels DeSSI provides

In [4]:
cols = []
labs = dessi_all_labels.T
labs.columns = dessi_all.columns
dessi_all = pd.concat([dessi_all, labs])
for c in dessi_all.columns:
    if c in ["married", "job", "salary", "password", "benefithealthinsurance"]:
        cols.append(c)
dessi_all[cols].iloc[[0,1,2,3,4,101], ::3]

,benefithealthinsurance,married,salary,password,benefithealthinsurance,salary,married,password,married,password,salary,benefithealthinsurance
0,no,Y,703535.21,TCC^WMu&afGU1F878m65nX6VB,no,703535.21,Y,TCC^WMu&afGU1F878m65nX6VB,Y,TCC^WMu&afGU1F878m65nX6VB,703535.21,no
1,yes,N,870468.74,AA*Tp1Hj+hbD!TVEPHe7B&8fh,yes,870468.74,N,AA*Tp1Hj+hbD!TVEPHe7B&8fh,N,AA*Tp1Hj+hbD!TVEPHe7B&8fh,870468.74,yes
2,yes,Y,829769.66,)P#Y8dXZVSh82vO$SzOx9MYe6,yes,829769.66,Y,)P#Y8dXZVSh82vO$SzOx9MYe6,Y,)P#Y8dXZVSh82vO$SzOx9MYe6,829769.66,yes
3,no,Y,665720.71,inLP8E@IkF084hdOjXebUCfp*,no,665720.71,Y,inLP8E@IkF084hdOjXebUCfp*,Y,inLP8E@IkF084hdOjXebUCfp*,665720.71,no
4,no,Y,953597.81,)g7tnIhvOJ+fvoq14^mmDD6kP,no,953597.81,Y,)g7tnIhvOJ+fvoq14^mmDD6kP,Y,)g7tnIhvOJ+fvoq14^mmDD6kP,953597.81,no
label,Other_data,Other_data,Other_data,Other_data,Other_data,Other_data,Other_data,Other_data,Other_data,Other_data,Other_data,Other_data


# Further investigations without relevant results

In [5]:
train = pd.read_csv("DeSSI_v2/train.csv")
dev = pd.read_csv("DeSSI_v2/dev.csv")
test = pd.read_csv("DeSSI_v2/test.csv")
train_labels = pd.read_csv("DeSSI_v2/train_labels.csv")
dev_labels = pd.read_csv("DeSSI_v2/dev_labels.csv")
test_labels = pd.read_csv("DeSSI_v2/test_labels.csv")

C:\Users\Luca\AppData\Local\Temp\ipykernel_1456\1766577438.py:1: DtypeWarning: Columns (62,107,161,241,255,324,449,503,686,720,807,812,889,934,1073,1113,1133,1154,1158,1242,1317,1333,1394,1408,1470,1553,1577,1674,1729,1762,1872,1913,1940,2045,2131,2139,2209,2288,2404,2436,2601,2627,2663,2779,2841,2960,2994,3061,3063,3080,3129,3136,3250,3319,3439,3464,3499,3520,3596,3637,3649,3748,3778,3787,3808,3902,3907,3917,3951,4009,4034,4037,4069,4129,4210,4218,4252,4325,4359,4386,4479,4646,4649,4652,4710,4936,4939,5013,5065,5069,5348,5352,5356,5399,5476,5702,5736,5889,5962,6072,6094,6107,6140,6159,6337,6360,6383,6407,6415,6423,6507,6555,6567,6590,6593,6689,6809,6815,6920,6995,7042,7127,7163,7229,7295,7301,7319,7321,7354,7399,7403,7465,7474,7593,7594,7694,7719,7798,7872,7979,8179,8298,8308,8473,8542,8565,8568,8570,8579,8633,8662,8707,8835,8846,8859,8899,8933,9024,9084,9126,9170,9184,9198,9251,9329,9412,9505,9588,9592,9610,9652,9803,9818,9907,9908,9952,9975,10073,10099,10136,10161,10359,10369,10407,

In [6]:
double_appearences = []
for c in test.columns:
    if c in train.columns:
        double_appearences.append(c)
len(double_appearences)

4116

In [7]:
for i in range(len(double_appearences)):
    if train[double_appearences[i]].value_counts().equals(test[double_appearences[i]].value_counts()):
        print(double_appearences[i])

married.6
married.9
datum_polnenje.1
activeto.3


In [8]:
train["married.6"]

0     1
1     1
2     0
3     1
4     1
     ..
95    1
96    1
97    0
98    1
99    1
Name: married.6, Length: 100, dtype: int64

In [9]:
test["married.6"]

0     1
1     1
2     0
3     1
4     0
     ..
95    0
96    0
97    0
98    0
99    1
Name: married.6, Length: 100, dtype: int64

Only four columns contain the same values between train and test set

In [10]:
count=0
label = []
for e, c in enumerate(train.columns):
    if "married" in c:
        count += 1  
        label.append(train_labels.iloc[e, :].values[0])
print(count)
print(pd.Series(label).value_counts())

74
Other_data    74
dtype: int64


In [11]:
count=0
label = []
for e, c in enumerate(test.columns):
    if "married" in c:
        count += 1  
        label.append(test_labels.iloc[e, :].values[0])
print(count)
print(pd.Series(label).value_counts())

35
Other_data    35
dtype: int64


Many columns have similar names, it could be that the model overfits based on the column names as similar column names have the same class  
The marriage status is a sensitive information but here only labeled as other data

Check how CASSED predicted wrong columns on uncleaned dataset (how many were correct even if the column was wrong?)

In [13]:
predictions = pd.read_csv("../../CASSED/old_predictions/CASSED_dessi/test.tsv",sep="\t")

dessi_labels_cleaned = pd.read_csv("dessi_cleaned/all_labels_cleaned.csv")
wrong_columns = pd.read_csv("dessi_cleaned/wrong_columns.csv")["0"]

def extract_prediction(i):
    list_extractor = re.compile(r"(\w+)\s*\(\d+(\.\d+)?\)")
    matches = list_extractor.findall(predictions.iloc[i, :].values[0])
    return [match[0] for match in matches]

def convert_predictions(predictions):
    column_names = []
    gold_list = []
    pred_list = []
    mismatch = []
    column_names.append(predictions.columns[0].split(". ")[0].strip())

    height = predictions.shape[0]

    for i in range(height):
        if 'Gold' in predictions.iloc[i, :].values[0][:10]:
            gold_list.append(extract_prediction(i))
        elif 'Pred' in predictions.iloc[i, :].values[0][:10]:
            pred_list.append(extract_prediction(i))
            if i == height-1:
                mismatch.append("No Mismatch")
                break
            if 'MISMATCH' in predictions.iloc[i+1, :].values[0][:20]:
                mismatch.append("Mismatch")
            else:
                mismatch.append("No Mismatch")
        elif 'MISMATCH' not in predictions.iloc[i, :].values[0][:20]:
            column_names.append(predictions.iloc[i, :].values[0].split('. ')[0].strip())
            


    df = pd.DataFrame({
        'Name': column_names,
        'Gold': gold_list,
        'Pred': pred_list,
        'Mismatch': mismatch
    })
    return df
df = convert_predictions(predictions)

In [14]:
wrong_columns_test = [int(a - (18816+6272)) for a in wrong_columns if a > 18816+6272]

In [15]:
df.iloc[wrong_columns_test,:].Mismatch.value_counts()

No Mismatch    153
Mismatch         9
Name: Mismatch, dtype: int64

Only 9 of the 162 wrong labeled columns were predicted wrong by CASSED 

# Detect Languages

Try to detect the languages in dessi dataset

In [16]:
dessi = pd.concat([train, dev, test], axis=1)
dessi.shape

(100, 31360)

In [17]:
dessi_labels = pd.concat([train_labels, dev_labels, test_labels],axis=0)
dessi_labels

,label
0,Date
1,Person
2,Geolocation
3,"Organization,Person"
4,Phone_number
...,...
6267,Geolocation
6268,Other_data
6269,NIN
6270,"Geolocation,Address"


In [18]:
dessi_transposed = dessi.T

dessi_transposed.insert(0, 'labels', dessi_labels.values.flatten())

dessi_with_labels = dessi_transposed.T

pd.set_option('display.max_columns', 100)
dessi_with_labels.head()

,axrlqygbssxu,firstname,lon,firm,fxvspjlbgrhm,jqfyjomipilf,location_id,phone_number,eosogjttsvtt,org_id,state,aevtkbjhhryq,customer_iban,departmentsid,switching_center_lat,relationship_status,phone_no,aoprdggmbtuz,wage,customer_email,country,family_email,countyname,vcqcujsmuukw,ywpjaeadqwwj,active,rrubbcpjrqht,kgazxjkrlqja,benefitdaycare,iinelubpyelh,gbhwunppuetx,lat,province,kdxrpomztopy,neighbourhood,jtklraxdukre,nkoyzuojoair,phone_ext,ssn,dqshzbqqvqdc,person_surname,lng,ukofatwbaafw,ijfsukxdjfhd,flikvuanjlbz,bndecuspucsy,datum_izvestaja,last_name,road_name,kozakymqwsyw,...,active.21,assisstantname.6,uebllxvjsdvb,city_name.1,cdomdvmaxlgy,mobilephonenumber.36,usmvdyjshjrb,sfiqeshcawas,id.14,personal_identification_number.10,ip_address.1,personal_code.6,active.22,udfybvtznvsn,first_name.28,ndvdzztxgkar,spouse.20,ceuisjeakbya,village.11,octslrhsuimb,staff_id.15,call_end_datetime.11,accountid.22,lastlogindateb.8,ktgjrkveeoeb,bwiugntwxfbp,npygcvxzyvwz,lkqncdpronyh,residence.2,city.24,cell_phone_number.20,vnpsaqxvggty,customer_id.13,monthly_earnings.17,eozjllawhlgb,spouse.21,accuracy_in_meters.15,customer_name.29,r identity,phone_ext.23,itittmxlzqjr,dadtysfqjbcs,zvneublxkuwy,personal_numerical_code.12,customer/contact,latdeg.12,genderid.7,ztyhgaddzlun,geo_latitude.18,date_account_closed.6
labels,Date,Person,Geolocation,"Organization,Person",Phone_number,"Address,Geolocation",Address,Phone_number,Person,Organization,GPE,Other_data,Other_data,Other_data,Geolocation,Other_data,Phone_number,Date,Other_data,Email,"Address,Geolocation",Phone_number,GPE,NIN,GPE,Other_data,Phone_number,Person,Other_data,Other_data,Other_data,Geolocation,GPE,NIN,Address,Other_data,Date,Phone_number,NIN,Other_data,Person,Geolocation,"Date,NIN",Phone_number,SWIFT/BIC,Phone_number,Date,Person,Address,Other_data,...,Other_data,Person,Organization,GPE,Other_data,Phone_number,Phone_number,Email,Organization,"NIN,Email",Other_data,"NIN,Person",Other_data,"Address,Email",Person,Email,Other_data,Address,Address,NIN,Other_data,Date,Other_data,Date,Date,Address,Date,Other_data,GPE,Address,Phone_number,GPE,Other_data,Other_data,Person,Other_data,Other_data,Person,Religion,Phone_number,Phone_number,"Address,Geolocation",Geolocation,NIN,"Person,Phone_number",Geolocation,Other_data,NIN,"Geolocation,Address",Date
0,4/12,Duča Grabowsky,39.4387199;-74.5492412,Marilyn,+55 (031) 4280-4254,-74.8062027,"40 Occom Ridge, Hanover, New Hampshire 03755, us",(071) 1428-1198,Rahman,Az Pneumatica,hungary,20603,21-0688878,35062,27.996344,Unknown,31 0020-5087,20090626,987881.94,danica-dereta3@xnet.hr,27.7753042,152691757/5,Jastrzębie-Zdrój,280 344 565,panama,1,0209981558,Salley,n,4238,Male,28.504554,Sondrio,464-24-9836,Costa del Turchese,48236.86,19700131,(470)4960902,50264871399,980696.7,Monica Shae Tanner,"39.3520555, -16.36384",045-05-6709,(051) 2074-9163,KPBGGBWT42Z,13840749598,19951202,Georgi,Seabrook Road,LOOK & FEEL,...,y,Ivy,sandton taxi cabs (pty) ltd,Hamburg,811021.61,+92 (165) 75254-48,18880977774,g-patel@becker-edwards.com,Heritage Production Services,emma-mehta@white-mcgrath.net,72.199.94.156,Ahmir Adriel Najera Masterson,n,m.wilson144@morrison-meyer.xyz,Anabelle Beckam Fredericks Ruth,azirovcic@leskovic.biz,yes,Westfield Avenue,Truro,111-72-1709,4733,12-1973,38577,20070804,30/04,"62 Möltner Straße - Via Meltina, Mölten - Melt...","January 5th, 2007",40546,Casnate con Bernate,Manchester,097 677-48-50,lithuania,7290,980260.86,Ransom Liam Worthy Starling,y,599088.57,Bryce Donaldson,Evangelical,31 4374 3014,334-314-6849x206,28.4060052,-16.76674,ZZ 55 65 56 T,Mae/ +44(0)1514960632,28.0778679; -74.6895013,1,ZZ 827141 T,"Strada Frena, Marèo - Enneberg - Marebbe, Tren...",05/1976
1,4/14,Lyubomir Vishinsky,39.4536535;-74.8164334,Broad run veterinary service,+23 (0511) 2695,"Grand Avenue rd. on number 84 in Manchester, N...","Via Milano - Mailandstraße, Bolzano - Bozen, I...",2576223255,Meyer,Visby,romania,20216,90-4343105,34675,-74.61295,Separated,+55 93 99082 4085,19800923,869968.7

In [19]:
dessi_labels.value_counts()

label                      
Other_data                     6561
Phone_number                   3547
Address                        2966
Date                           2319
Person                         2234
GPE                            2172
NIN                            1706
Geolocation                    1635
Organization                   1607
Email                          1533
NIN,Date                        241
SWIFT/BIC                       240
Organization,Person             240
NIN,Person                      230
Phone_number,NIN                226
Person,Organization             224
Date,NIN                        221
Person,Email                    220
Person,NIN                      218
Geolocation,Address             218
Email,Phone_number              217
NIN,Phone_number                217
Email,Address                   217
IBAN                            213
NIN,Email                       210
Email,Person                    209
Address,Geolocation             205


In [20]:
DetectorFactory.seed = 0

df = pd.DataFrame(dessi)

# Function to detect language of a text
def detect_language(text):
    try:
        return detect(text)
    except LangDetectException:
        return "unknown"

# Apply the function to the DataFrame
languages = set()
for i in range(len(df.columns)):
    if df.iloc[:,i].dtype == 'object':
        print(i)
        print(set(df.iloc[:,i].apply(detect_language)))
        languages.update(set(df.iloc[:,i].apply(detect_language)))

# Display the results
print(df)


0
{'unknown'}
1
{'de', 'so', 'it', 'et', 'hu', 'tl', 'fi', 'sk', 'sv', 'id', 'sq', 'pl', 'hr', 'lt', 'en', 'sl', 'ro', 'no', 'af', 'pt', 'es', 'tr'}
2
{'unknown'}
3
{'de', 'so', 'fr', 'ca', 'it', 'tl', 'fi', 'sk', 'sv', 'id', 'sw', 'nl', 'hr', 'lt', 'en', 'da', 'sl', 'cs', 'ro', 'no', 'cy', 'af', 'pt', 'es', 'tr'}
4
{'so', 'unknown'}
5
{'de', 'unknown', 'pl', 'fi', 'fr', 'ca', 'it', 'en', 'nl'}
6
{'cs', 'de', 'no', 'fr', 'pl', 'it', 'ca', 'lt', 'sv', 'af', 'en', 'es', 'da', 'nl'}
7
{'so', 'ca', 'unknown'}
8
{'de', 'so', 'fr', 'it', 'ca', 'et', 'tl', 'fi', 'sk', 'sv', 'id', 'sw', 'nl', 'pl', 'hr', 'lt', 'en', 'da', 'ro', 'no', 'lv', 'cy', 'af', 'pt', 'es', 'tr'}
9
{'de', 'so', 'fr', 'ca', 'it', 'et', 'hu', 'tl', 'fi', 'id', 'nl', 'hr', 'en', 'da', 'cs', 'ro', 'lv', 'cy', 'af', 'pt', 'es'}
10
{'de', 'so', 'ca', 'it', 'hu', 'vi', 'tl', 'fi', 'sv', 'id', 'sw', 'nl', 'pl', 'hr', 'lt', 'en', 'sl', 'cs', 'ro', 'no', 'lv', 'cy', 'pt', 'tr'}
12
{'unknown'}
15
{'id', 'en', 'it', 'pl'}


KeyboardInterrupt: 

Forced a keyboard interrupt as in columns like name or email it detects many languages. It is hard to filter these columns automatically.